In [ ]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./jointopt/joint_dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_constraint.jl")
include("./trajopt/scaling.jl")

In [ ]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/xuQYZ_unicycle_0328" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"]
Qnom = my_dict["Q"]
Ynom = my_dict["Y"]
Znom = my_dict["Z"]
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [ ]:
ix = 3
iu = 2
iϕ = 2
iv = 2
iψ = iϕ*iv
iμ = iψ
@assert size(xnom,2) - 1 == N

In [ ]:
Go = [1 2;1 2;0 0]
G = kron(Go,ones(1,iv))
Cv = [0 0 1;0 0 0] 
Dvu = [0 0;1 0]

In [ ]:
dynamics = Unicycle()
alpha = 0.7
DLMI = LinearQY(alpha,ix,iu)

In [ ]:
idx = 4
Aq,Bq,Sq,Ax,Bu = diff(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx])

In [ ]:
eps_x = Diagonal{Float64}(I, ix)
eps_u = Diagonal{Float64}(I, iu)
Fx = zeros(iq,ix)
Fu = zeros(iq,iu)
h = 2^(-18)
for i in 1:ix
    Fx[:,i] .= (
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx]+h*eps_x[:,i],unom[:,idx]) -
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx]-h*eps_x[:,i],unom[:,idx])
    ) / (2*h)
end
for i in 1:iu
    Fu[:,i] .= (
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx]+h*eps_u[:,i]) -
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx]-h*eps_u[:,i])
    ) / (2*h)
end
# for i in 1:iu
#     Fu[:,i] = (forward(model,x,u+h*eps_u[:,i]) - forward(model,x,u-h*eps_u[:,i])) / (2*h)
# end

In [ ]:
sum(abs.(Ax - Fx))

In [ ]:
sum(abs.(Bu-Fu))

In [ ]:
eps_x = Diagonal{Float64}(I, iq)
eps_u = Diagonal{Float64}(I, iy)
Fq = zeros(iq,iq)
Fy = zeros(iq,iy)
h = 2^(-18)
for i in 1:iq
    Fq[:,i] .= (
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx])+h*eps_x[:,i],vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx]) -
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx])-h*eps_x[:,i],vec(Ynom[:,:,idx]),vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx])
    ) / (2*h)
end
for i in 1:iy
    Fy[:,i] .= (
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx])+h*eps_u[:,i],vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx]) -
    forward(DLMI,dynamics,
    vec(Qnom[:,:,idx]),vec(Ynom[:,:,idx])-h*eps_u[:,i],vec(Znom[:,:,idx]),
    xnom[:,idx],unom[:,idx])
    ) / (2*h)
end

In [ ]:
sum(abs.(Aq - Fq))

In [ ]:
sum(abs.(Bq - Fy))

In [ ]:
a = 1:10
a[end]

In [ ]:
include("./jointopt/joint_dynamics.jl")

In [ ]:
A,Bm,Bp,Aq,Ax,Bum,Bup,Bym,Byp,Bzm,Bzp,zf,zF,x_prop,q_prop = discretize_foh(DLMI,dynamics,
    Qnom[:,:,1:N],Ynom,Znom,xnom[:,1:N],unom,dtnom)

In [ ]:
print_jl(Qnom)